In [1]:
import re

import pandas as pd

## Load

In [2]:
# data = pd.read_excel("/opt/ml/data/total.xlsx")
# data = data.drop(columns="Unnamed: 0")
# data = data.drop(data[data["text"].isnull()].index)

# data

In [3]:
data = pd.read_csv("/opt/ml/data/poems_v1.csv")
data

,genre,title,text,length
0,시,양을 찾아서,마침내 양은 사라졌다\n\n한 의식을 잃고서 나는 은주발에 담긴 눈이 되고 싶었다\...,479
1,시조,금빛 질경이,"흙바람 길을 튼다, 길섶에 씨방 연다\n비에 젖은 잎새 위에 숨 고르는 햇살 한 줌...",203
2,시,폐가를 어루만지다,"""허물어지는 것은 새것을 위한 눈부신 산화\n나는 철거될 농가의 마룻바닥에 가만 귀...",481
3,시조,"문장부호, 느루찍다2",점 하나 못 챙긴 채 빈 공간에 갇히는 날\n말없음표 끌어다가 어질머리 잠재우고\n...,272
4,시,대봉,파르티잔들이\n노모의 흐린 눈에 가을을 찔러 넣는다.\n턱밑에 은빛 강물을 가두고 ...,211
...,...,...,...,...
16371,시조,환절기를 걷다,1.\n\n벚꽃은 흩날리고 떠나는 너의 뒷모습은\n\n출항하는 바다에 비친 등불을 ...,213
16372,시조,벗고싶은 봄,코로나 바이러스 마스크 5부제가\n\n담쟁이 넝쿨처럼 담 벽을 둘러쳐도\n\n빈손을...,263
16373,시조,숫돌을 읽다,고향 집 우물가에 등 굽은 검은 숫돌\n\n지문이 없어지듯 닳고 닳은 오목가슴\n\...,252
16374,시조,부여,유적 같은 도시에서 유서 같은 시를 쓴다\n\n아버지와 어색하다 식탁이 너무 넓다\...,197


## Process

In [4]:
doublespace_pattern = re.compile("[\f\r\t\v]+")
repeatchars_pattern = re.compile("(\w)\\1{3,}")


def repeat_normalize(sent, num_repeats=2):
    if num_repeats > 0:
        sent = repeatchars_pattern.sub("\\1" * num_repeats, sent)
    sent = doublespace_pattern.sub(" ", sent)
    return sent.strip()


def preprocess(text):
    # 문제를 일으킬 수 있는 문자 제거
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char])

    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0": " "}
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])

    # 이메일 제거
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "[이메일]", text).strip()

    # "#문자" 형식 어절 제거
    text = re.sub(r"#\S+", "", text).strip()

    # "@문자" 형식 어절 제거
    text = re.sub(r"@\w+", "", text).strip()

    text = re.sub("[0-9]+\.", "", text).strip()

    # URL 제거
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "[웹주소]", text).strip()
    text = re.sub(r"pic\.(\w+\.)+\S*", "[웹주소]", text).strip()

    # 뉴스 저작권 관련 텍스트 제거
    re_patterns = [
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))",
    ]

    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()

    # 뉴스 내 포함된 이미지에 대한 레이블 제거
    text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()

    # 중복 문자 처리
    text = repeat_normalize(text, num_repeats=2).strip()

    # 문제를 일으킬 수 있는 구두점 치환
    punct_mapping = {
        "‘": "'",
        "₹": "e",
        "´": "'",
        "°": "",
        "€": "e",
        "™": "tm",
        "√": " sqrt ",
        "×": "x",
        "²": "2",
        "—": "-",
        "–": "-",
        "’": "'",
        "_": "-",
        "`": "'",
        "“": '"',
        "”": '"',
        "“": '"',
        "£": "e",
        "∞": "infinity",
        "θ": "theta",
        "÷": "/",
        "α": "alpha",
        "•": ".",
        "à": "a",
        "−": "-",
        "β": "beta",
        "∅": "",
        "³": "3",
        "π": "pi",
        "*": "",
    }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])

    # 연속된 공백 치환
    text = re.sub(r"[\f\r\t\v]+", " ", text).strip()
    text = re.sub("\n ", "\n", text)
    text = re.sub("\n{3,}", "\n\n", text)

    return text


In [5]:
d = data.to_dict()
for k, text in d["text"].items():
    d["text"][k] = preprocess(text)

data = pd.DataFrame(d)
data.to_csv("/opt/ml/data/poems_v1_processed.csv")
data

,genre,title,text,length
0,시,양을 찾아서,마침내 양은 사라졌다\n\n한 의식을 잃고서 나는 은주발에 담긴 눈이 되고 싶었다\...,479
1,시조,금빛 질경이,"흙바람 길을 튼다, 길섶에 씨방 연다\n비에 젖은 잎새 위에 숨 고르는 햇살 한 줌...",203
2,시,폐가를 어루만지다,"""허물어지는 것은 새것을 위한 눈부신 산화\n나는 철거될 농가의 마룻바닥에 가만 귀...",481
3,시조,"문장부호, 느루찍다2",점 하나 못 챙긴 채 빈 공간에 갇히는 날\n말없음표 끌어다가 어질머리 잠재우고\n...,272
4,시,대봉,파르티잔들이\n노모의 흐린 눈에 가을을 찔러 넣는다.\n턱밑에 은빛 강물을 가두고 ...,211
...,...,...,...,...
16371,시조,환절기를 걷다,벚꽃은 흩날리고 떠나는 너의 뒷모습은\n\n출항하는 바다에 비친 등불을 닮았다\n\...,213
16372,시조,벗고싶은 봄,코로나 바이러스 마스크 5부제가\n\n담쟁이 넝쿨처럼 담 벽을 둘러쳐도\n\n빈손을...,263
16373,시조,숫돌을 읽다,고향 집 우물가에 등 굽은 검은 숫돌\n\n지문이 없어지듯 닳고 닳은 오목가슴\n\...,252
16374,시조,부여,유적 같은 도시에서 유서 같은 시를 쓴다\n\n아버지와 어색하다 식탁이 너무 넓다\...,197


## Collect specific genre

In [6]:
# poem = data[data["genre"].apply(lambda x: x in ["시", "동시", "시조"])]
# poem = poem.reset_index().drop(columns="index")
# poem.to_csv("/opt/ml/data/poem.csv")
# poem